In [1]:
input_dir = '/home/matteo/ctasoft/resources'
gamma_diffuse = 'gamma_20deg_0deg_run101___cta-prod3-lapalma3-2147m-LaPalma.simtel.gz'
sys_path = '/home/matteo/ctasoft/'
prepared_data_dir = 'prepared_data/output'

In [2]:
import sys
import os
sys.path.append(sys_path + "cta-benchmarks/Preparation/parameters_search")  # path contains python_file.py
from quality_parameters_function import iterate_param, create_possible_parameters

/home/matteo/miniconda3/envs/cta-benchmarks/lib/python3.6/site-packages/corsikaio/subblocks/dtypes.py:20: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dict(**dt))


In [3]:
"""
script to find the optimized levels of cleaning for each image of an event.

Data are saved in the script's path in a .npz file

"""

from ctapipe.utils.datasets import find_in_path
from ctapipe.io import event_source
from ctapipe.calib import CameraCalibrator
import numpy as np

possible_cleaning_levels = create_possible_parameters()

quality_param_list = []
hillas_containers = {}

data = []

In [4]:
prepared_data_dir = '../../../' + prepared_data_dir

if not os.path.exists(prepared_data_dir):
    os.makedirs(prepared_data_dir, exist_ok=True)

In [5]:
input_url = find_in_path(gamma_diffuse, input_dir)
events = event_source(input_url, max_events=1)

calibrator = CameraCalibrator()

In [6]:
for number_event, event in enumerate(events):
    hillas_containers.clear()
    calibrator(event)
    data.append((number_event, event.mc.energy, event.inst.subarray.tel_coords))

    for telescope_id, dl1 in event.dl1.tel.items():
        quality_param_list.clear()

        image = dl1.image
        camera = event.inst.subarray.tels[telescope_id].camera

        optimized_cleaning_level = iterate_param(possible_cleaning_levels, event, camera, image,
                                                 telescope_id, quality_param_list, hillas_containers)

        data.append((camera.cam_id, telescope_id,
                     optimized_cleaning_level))

np.savez_compressed(prepared_data_dir+'/parameters_' + gamma_diffuse + '.npz', data=data)

loaded = np.load(prepared_data_dir+'/parameters_' + gamma_diffuse + '.npz', allow_pickle=True)

data_loaded = loaded['data']

print(data_loaded)

[[0 <Quantity 0.08440065 TeV>
  <SkyCoord (GroundFrame): (x, y, z) in m
    [( -70.05000305,   -7.23000002,  56.        ),
     ( -12.30000019,   71.29000092,  47.        ),
     (  71.81999969,    8.19999981,  43.        ),
     (   9.31000042,  -68.06999969,  50.        ),
     ( -70.04000092,   -7.23000002,  54.        ),
     ( -34.36999893,  110.98000336,  43.        ),
     (  76.18000031,   95.33999634,  39.70000076),
     (  31.80999947,  -19.70000076,  43.        ),
     (   0.        ,    0.        ,  43.        ),
     ( 180.32000732,   28.52000046,  20.        ),
     ( -22.53000069,  196.8999939 ,  27.        ),
     ( 100.94000244,  124.51000214,  31.        ),
     ( 169.42999268, -125.27999878,  37.        ),
     (  20.55999947, -199.05000305,  48.        ),
     ( -97.34999847, -126.30000305,  57.        ),
     (-198.58999634,  -26.45000076,  59.        ),
     (-168.78999329,  124.22000122,  52.        ),
     (  66.38999939,  341.44000244,  16.        ),
     ( 246